# Spatial Taylor Expansion
Instead of computing the term-by-term Taylor Expansion for pCO$_{2}$ over the whole region, we now move on to compute the Taylor Expansion for each grid cell in the region. This is similar to the spatial plots we made for spatial correlations. Hopefully it can give us an idea of the patterns that may emerge in a given region. The goal is to make this into a standalone .py script.

In [581]:
import numpy as np
import pandas as pd
import xarray as xr
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.basemap import Basemap

# CalCS Practice


In [515]:
def create_term(ds, var):
    """
    Input the post-regression spatial dataset and a variable name. This will return a DataArray that has
    computed the taylor expansion term value and has named the DataArray properly.
    """
    term = ds['sens_' + var] * ds[var + '_slopes']
    term.name = var + '_term'
    return term

In [426]:
fileDir = '/glade/u/home/rbrady/projects/EBUS_BGC_Variability/data/processed/calcs/pco2_taylor_spatial/calcs-pco2-taylor-spatial-vs-nino34-smoothed-pRemoved.nc'
ds = xr.open_dataset(fileDir)

In [561]:
# Term-by-term Taylor Expansion
dic_term = create_term(ds, 'DIC')
alk_term = create_term(ds, 'ALK')
sst_term = create_term(ds, 'SST')
salt_term = create_term(ds, 'SALT')

# Total terms
approx_total = dic_term + alk_term + sst_term + salt_term
approx_total.name = 'approx_total'
model_total = ds['pCO2SURF_slopes']
model_total.name = 'model_total'

# Only include grid cells where every term has p < 0.05
conditions = (~np.isnan(dic_term)) & (~np.isnan(alk_term)) & \
             (~np.isnan(sst_term)) & (~np.isnan(salt_term))

# Create new datset for these.
taylor_expansion = xr.merge([dic_term, alk_term, sst_term, salt_term, approx_total, model_total])
taylor_expansion = taylor_expansion.where(conditions)

# Visualization

In [582]:
def visual_bounds(EBU):
    if EBU == "CalCS":
        lat1 = 25
        lat2 = 43
        lon1 = -133
        lon2 = -110
    elif EBU == 'HumCS':
        lat1 = -20
        lat2 = -2
        lon1 = -90
        lon2 = -70
    elif EBU == 'CanCS':
        lat1 = 10
        lat2 = 28
        lon1 = -25
        lon2 = -10
    elif EBU == 'BenCS':
        lat1 = -35
        lat2 = -15
        lon1 = 5
        lon2 = 20
    return lat1,lat2,lon1,lon2

In [ ]:
def setup_map(ax, lat1, lat2, lon1, lon2):
    m = Basemap(projection='merc',
                lat_0=(lat1+lat2)/2,
                lon_0=(lon1+lon2)/2,
                #width=(2000*1000),
                #height=(2000*1000),
                llcrnrlon=lon1,
                urcrnrlon=lon2,
                llcrnrlat=lat1,
                urcrnrlat=lat2,
                fix_aspect=False,
                resolution='i')
    m.drawcoastlines(color='#D3D3D3')
    #m.drawcountries()
    #m.drawstates()
    m.fillcontinents(color='#D3D3D3')
    m.drawmapboundary(fill_color='white')
    m.drawparallels(np.arange(lat1, lat2+1, 3),
                   labels=[False,True,False,False], size=20,
                   color='w')
    m.drawmeridians(np.arange(lon1, lon2+1, 3),
                   labels=[False, False, False, False], size='x-large',
                   color='w')
    return m

In [579]:
fig = plt.figure(figsize=(16,8))


taylor_expansion

<xarray.Dataset>
Dimensions:       (ensemble: 34, nlat: 46, nlon: 40)
Coordinates:
    z_t           float64 500.0
  * nlat          (nlat) int64 265 266 267 268 269 270 271 272 273 274 275 ...
  * nlon          (nlon) int64 226 227 228 229 230 231 232 233 234 235 236 ...
    TLAT          (nlat, nlon) float64 25.14 25.12 25.09 25.07 25.04 25.02 ...
    TLONG         (nlat, nlon) float64 215.0 216.1 217.3 218.4 219.5 220.6 ...
    TAREA         (nlat, nlon) float64 nan nan nan nan nan nan nan nan nan ...
  * ensemble      (ensemble) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 ...
Data variables:
    DIC_term      (nlat, nlon, ensemble) float64 nan nan nan nan nan nan nan ...
    ALK_term      (nlat, nlon, ensemble) float64 nan nan nan nan nan nan nan ...
    SST_term      (nlat, nlon, ensemble) float64 nan nan nan nan nan nan nan ...
    SALT_term     (nlat, nlon, ensemble) float64 nan nan nan nan nan nan nan ...
    approx_total  (nlat, nlon, ensemble) float64 nan nan nan nan nan n